Copyright (c) MONAI Consortium  
Licensed under the Apache License, Version 2.0 (the "License");  
you may not use this file except in compliance with the License.  
You may obtain a copy of the License at  
&nbsp;&nbsp;&nbsp;&nbsp;http://www.apache.org/licenses/LICENSE-2.0  
Unless required by applicable law or agreed to in writing, software  
distributed under the License is distributed on an "AS IS" BASIS,  
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
See the License for the specific language governing permissions and  
limitations under the License.

# MAISI Inference Tutorial

This tutorial illustrates how to use trained MAISI model and codebase to generate synthetic 3D images and paired masks.

## Setup environment

In [1]:
!python -c "import monai" || pip install -q "monai-weekly[nibabel, tqdm]"
!python -c "import matplotlib" || pip install -q matplotlib
%matplotlib inline

## Setup imports

In [2]:
import argparse
import json
import os
import tempfile

import monai
import torch
from monai.apps import download_url
from monai.config import print_config
from monai.transforms import LoadImage, Orientation
from monai.utils import set_determinism
from scripts.sample import LDMSampler, check_input
from scripts.utils import define_instance, load_autoencoder_ckpt, load_diffusion_ckpt
from scripts.utils_plot import find_label_center_loc, get_xyz_plot, show_image

print_config()

MONAI version: 1.4.0rc1+1.g4877767c
Numpy version: 1.26.0
Pytorch version: 2.3.1+cu121
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 4877767cf92649a38ffda0fc590f2b92ba59f019
MONAI __file__: /localhome/<username>/.local/lib/python3.10/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.11
ITK version: 5.4.0
Nibabel version: 5.2.1
scikit-image version: 0.23.2
scipy version: 1.14.0
Pillow version: 10.4.0
Tensorboard version: 2.17.0
gdown version: 5.2.0
TorchVision version: 0.18.1+cu121
tqdm version: 4.66.4
lmdb version: 1.5.1
psutil version: 6.0.0
pandas version: 2.2.2
einops version: 0.8.0
transformers version: 4.40.2
mlflow version: 2.15.1
pynrrd version: 1.0.0
clearml version: 1.16.3

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



## Setup data directory
You can specify a directory with the `MONAI_DATA_DIRECTORY` environment variable.
This allows you to save results and reuse downloads.
If not specified a temporary directory will be used.

In [3]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
if directory is not None:
    os.makedirs(directory, exist_ok=True)
root_dir = tempfile.mkdtemp() if directory is None else directory

# TODO: remove the `files` after the files are uploaded to the NGC
files = [
    {
        "path": "models/autoencoder_epoch273.pt",
        "url": "https://drive.google.com/file/d/1jQefG0yJPzSvTG5rIJVHNqDReBTvVmZ0/view?usp=drive_link",
    },
    {
        "path": "models/input_unet3d_data-all_steps1000size512ddpm_random_current_inputx_v1.pt",
        "url": "https://drive.google.com/file/d/1FtOHBGUF5dLZNHtiuhf5EH448EQGGs-_/view?usp=sharing",
    },
    {
        "path": "models/controlnet-20datasets-e20wl100fold0bc_noi_dia_fsize_current.pt",
        "url": "https://drive.google.com/file/d/1izr52Whkk56OevNTk2QzI86eJV9TTaLk/view?usp=sharing",
    },
    {
        "path": "models/mask_generation_autoencoder.pt",
        "url": "https://drive.google.com/file/d/1FzWrpv6ornYUaPiAWGOOxhRx2P9Wnynm/view?usp=drive_link",
    },
    {
        "path": "models/mask_generation_diffusion_unet.pt",
        "url": "https://drive.google.com/file/d/11SA9RUZ6XmCOJr5v6w6UW1kDzr6hlymw/view?usp=drive_link",
    },
    {
        "path": "configs/candidate_masks_flexible_size_and_spacing_3000.json",
        "url": "https://drive.google.com/file/d/1yMkH-lrAsn2YUGoTuVKNMpicziUmU-1J/view?usp=sharing",
    },
    {
        "path": "configs/all_anatomy_size_condtions.json",
        "url": "https://drive.google.com/file/d/1AJyt1DSoUd2x2AOQOgM7IxeSyo4MXNX0/view?usp=sharing",
    },
    {
        "path": "datasets/all_masks_flexible_size_and_spacing_3000.zip",
        "url": "https://drive.google.com/file/d/16MKsDKkHvDyF2lEir4dzlxwex_GHStUf/view?usp=sharing",
    },
]

for file in files:
    file["path"] = file["path"] if "datasets/" not in file["path"] else os.path.join(root_dir, file["path"])
    download_url(url=file["url"], filepath=file["path"])

2024-08-14 17:06:54,446 - INFO - Expected md5 is None, skip md5 check for file models/autoencoder_epoch273.pt.
2024-08-14 17:06:54,447 - INFO - File exists: models/autoencoder_epoch273.pt, skipped downloading.
2024-08-14 17:06:54,448 - INFO - Expected md5 is None, skip md5 check for file models/input_unet3d_data-all_steps1000size512ddpm_random_current_inputx_v1.pt.
2024-08-14 17:06:54,448 - INFO - File exists: models/input_unet3d_data-all_steps1000size512ddpm_random_current_inputx_v1.pt, skipped downloading.
2024-08-14 17:06:54,449 - INFO - Expected md5 is None, skip md5 check for file models/controlnet-20datasets-e20wl100fold0bc_noi_dia_fsize_current.pt.
2024-08-14 17:06:54,449 - INFO - File exists: models/controlnet-20datasets-e20wl100fold0bc_noi_dia_fsize_current.pt, skipped downloading.
2024-08-14 17:06:54,450 - INFO - Expected md5 is None, skip md5 check for file models/mask_generation_autoencoder.pt.
2024-08-14 17:06:54,450 - INFO - File exists: models/mask_generation_autoencoder

Downloading...
From (original): https://drive.google.com/uc?id=16MKsDKkHvDyF2lEir4dzlxwex_GHStUf
From (redirected): https://drive.google.com/uc?id=16MKsDKkHvDyF2lEir4dzlxwex_GHStUf&confirm=t&uuid=90c41b32-cfaf-4274-a0ec-dbd6408cf638
To: /tmp/tmpe4zcopul/all_masks_flexible_size_and_spacing_3000.zip
100%|██████████████████████████████████████| 9.03G/9.03G [01:54<00:00, 79.1MB/s]

2024-08-14 17:08:52,675 - INFO - Downloaded: /tmp/tmpr4d6e0ww/datasets/all_masks_flexible_size_and_spacing_3000.zip
2024-08-14 17:08:52,675 - INFO - Expected md5 is None, skip md5 check for file /tmp/tmpr4d6e0ww/datasets/all_masks_flexible_size_and_spacing_3000.zip.


## Read in environment setting, including data directory, model directory, and output directory

The information for data directory, model directory, and output directory are saved in ./configs/environment.json

In [4]:
args = argparse.Namespace()

environment_file = "./configs/environment.json"
env_dict = json.load(open(environment_file, "r"))
for k, v in env_dict.items():
    # Update the path to the downloaded dataset in MONAI_DATA_DIRECTORY
    val = v if "datasets/" not in v else os.path.join(root_dir, v)
    setattr(args, k, val)
    print(f"{k}: {val}")
print("Global config variables have been loaded.")

output_dir: output
trained_autoencoder_path: models/autoencoder_epoch273.pt
trained_diffusion_path: models/input_unet3d_data-all_steps1000size512ddpm_random_current_inputx_v1.pt
trained_controlnet_path: models/controlnet-20datasets-e20wl100fold0bc_noi_dia_fsize_current.pt
trained_mask_generation_autoencoder_path: models/mask_generation_autoencoder.pt
trained_mask_generation_diffusion_path: models/mask_generation_diffusion_unet.pt
all_mask_files_base_dir: /tmp/tmpr4d6e0ww/datasets/all_masks_flexible_size_and_spacing_3000
all_mask_files_json: ./configs/candidate_masks_flexible_size_and_spacing_3000.json
all_anatomy_size_conditions_json: ./configs/all_anatomy_size_condtions.json
label_dict_json: ./configs/label_dict.json
label_dict_remap_json: ./configs/label_dict_124_to_132.json
Global config variables have been loaded.


## Read in configuration setting, including network definition, body region and anatomy to generate, etc.

The information for the inference input, like body region and anatomy to generate, is stored in "./configs/config_infer.json". Please refer to README.md for the details.

In [5]:
config_file = "./configs/config_maisi.json"
config_dict = json.load(open(config_file, "r"))
for k, v in config_dict.items():
    setattr(args, k, v)

# check the format of inference inputs
config_infer_file = "./configs/config_infer.json"
config_infer_dict = json.load(open(config_infer_file, "r"))
for k, v in config_infer_dict.items():
    setattr(args, k, v)
    print(f"{k}: {v}")

check_input(
    args.body_region,
    args.anatomy_list,
    args.label_dict_json,
    args.output_size,
    args.spacing,
    args.controllable_anatomy_size,
)
latent_shape = [args.latent_channels, args.output_size[0] // 4, args.output_size[1] // 4, args.output_size[2] // 4]
print("Network definition and inference inputs have been loaded.")

num_output_samples: 1
body_region: ['abdomen']
anatomy_list: ['liver', 'hepatic tumor']
controllable_anatomy_size: []
num_inference_steps: 1000
mask_generation_num_inference_steps: 1000
output_size: [256, 256, 256]
image_output_ext: .nii.gz
label_output_ext: .nii.gz
spacing: [1.5, 1.5, 2.0]
autoencoder_sliding_window_infer_size: [48, 48, 48]
autoencoder_sliding_window_infer_overlap: 0.25
Network definition and inference inputs have been loaded.


## Set deterministic training for reproducibility

In [6]:
set_determinism(seed=0)
args.random_seed = 0

## Initialize networks and noise scheduler, then load the trained model weights.

The networks and noise scheduler are defined in `config_maisi.json`. We will read them in and load the model weights.

In [10]:
noise_scheduler = define_instance(args, "noise_scheduler")
mask_generation_noise_scheduler = define_instance(args, "mask_generation_noise_scheduler")

device = torch.device("cuda")

autoencoder = define_instance(args, "autoencoder_def").to(device)
checkpoint_autoencoder = load_autoencoder_ckpt(args.trained_autoencoder_path)
autoencoder.load_state_dict(checkpoint_autoencoder)

diffusion_unet = define_instance(args, "diffusion_unet_def").to(device)
checkpoint_diffusion_unet = torch.load(args.trained_diffusion_path)
new_dict = load_diffusion_ckpt(diffusion_unet.state_dict(), checkpoint_diffusion_unet["unet_state_dict"])
diffusion_unet.load_state_dict(new_dict, strict=True)
scale_factor = checkpoint_diffusion_unet["scale_factor"].to(device)

controlnet = define_instance(args, "controlnet_def").to(device)
checkpoint_controlnet = torch.load(args.trained_controlnet_path)
new_dict = load_diffusion_ckpt(controlnet.state_dict(), checkpoint_controlnet["controlnet_state_dict"])
monai.networks.utils.copy_model_state(controlnet, diffusion_unet.state_dict())
controlnet.load_state_dict(new_dict, strict=True)

mask_generation_autoencoder = define_instance(args, "mask_generation_autoencoder_def").to(device)
checkpoint_mask_generation_autoencoder = load_autoencoder_ckpt(args.trained_mask_generation_autoencoder_path)
mask_generation_autoencoder.load_state_dict(checkpoint_mask_generation_autoencoder)

mask_generation_diffusion_unet = define_instance(args, "mask_generation_diffusion_def").to(device)
checkpoint_mask_generation_diffusion_unet = torch.load(args.trained_mask_generation_diffusion_path)
mask_generation_diffusion_unet.load_old_state_dict(checkpoint_mask_generation_diffusion_unet)
mask_generation_scale_factor = args.mask_generation_scale_factor

print("All the trained model weights have been loaded.")

2024-08-14 17:12:39,907 - INFO - 'dst' model updated: 158 of 206 variables.
All the trained model weights have been loaded.


## Define the LDM Sampler, which contains functions that will perform the inference.

In [11]:
ldm_sampler = LDMSampler(
    args.body_region,
    args.anatomy_list,
    args.all_mask_files_json,
    args.all_anatomy_size_conditions_json,
    args.all_mask_files_base_dir,
    args.label_dict_json,
    args.label_dict_remap_json,
    autoencoder,
    diffusion_unet,
    controlnet,
    noise_scheduler,
    scale_factor,
    mask_generation_autoencoder,
    mask_generation_diffusion_unet,
    mask_generation_scale_factor,
    mask_generation_noise_scheduler,
    device,
    latent_shape,
    args.mask_generation_latent_shape,
    args.output_size,
    args.output_dir,
    args.controllable_anatomy_size,
    image_output_ext=args.image_output_ext,
    label_output_ext=args.label_output_ext,
    spacing=args.spacing,
    num_inference_steps=args.num_inference_steps,
    mask_generation_num_inference_steps=args.mask_generation_num_inference_steps,
    random_seed=args.random_seed,
    autoencoder_sliding_window_infer_size=args.autoencoder_sliding_window_infer_size,
    autoencoder_sliding_window_infer_overlap=args.autoencoder_sliding_window_infer_overlap,
)

## Perform the inference
**Time cost:** It will take around 80s to generate a [256,256,256] mask, and another 105s to generate the corresponding [256,256,256] image on one A100. The time cost per mask is fixed; time cost per image is roughly linear to the output size. i.e., if we want to generate a [512,512,512] image/mask pair, it will take around 80s to generate a [256,256,256] mask and then resample it to a [512,512,512] mask. The time cost for [512,512,512] image generatation will be 8 times longer, around 8x105s~14min. 

**GPU memory:** Decoding the latents into image with autoencoder is the step that consumes the largest GPU memory. The mask generation takes 34G. For image generation, to save GPU memory, we use sliding window inference for the autoencoder when the `output_size` is large. So the GPU memory for image generation is not strictly linear to the output_size. For [256,256,256], it takes 34G. For [512,512,512], it takes 67G.

In [ ]:
print(f"The generated image/mask pairs will be saved in {args.output_dir}.")
output_filenames = ldm_sampler.sample_multiple_images(args.num_output_samples)
print("MAISI image/mask generation finished")

The generated image/mask pairs will be saved in output.
Extracting /tmp/tmpr4d6e0ww/datasets/all_masks_flexible_size_and_spacing_3000.zip to /tmp/tmpr4d6e0ww/datasets
2024-08-14 17:12:42,609 - INFO - Writing into directory: /tmp/tmpr4d6e0ww/datasets.
Unzipped /tmp/tmpr4d6e0ww/datasets/all_masks_flexible_size_and_spacing_3000.zip to /tmp/tmpr4d6e0ww/datasets/all_masks_flexible_size_and_spacing_3000.
augmenting liver tumor


 29%|████████████████████▌                                                 | 293/1000 [00:28<01:07, 10.53it/s]

## Visualize the results


In [ ]:
visualize_image_filename = output_filenames[0][0]
visualize_mask_filename = output_filenames[0][1]
print(f"Visualizing {visualize_image_filename} and {visualize_mask_filename}...")

# load image/mask pairs
loader = LoadImage(image_only=True, ensure_channel_first=True)
orientation = Orientation(axcodes="RAS")
image_volume = orientation(loader(visualize_image_filename))
mask_volume = orientation(loader(visualize_mask_filename)).to(torch.uint8)

# visualize for CT HU intensity between [-200, 500]
image_volume = torch.clip(image_volume, -200, 500)
image_volume = image_volume - torch.min(image_volume)
image_volume = image_volume / torch.max(image_volume)

# create a random color map for mask visualization
colorize = torch.clip(torch.cat([torch.zeros(3, 1, 1, 1), torch.randn(3, 200, 1, 1)], 1), 0, 1)
target_class_index = 1

# find center voxel location for 2D slice visualization
center_loc_axis = find_label_center_loc(torch.flip(mask_volume[0, ...] == target_class_index, [-3, -2, -1]))

# visualization
vis_mask = get_xyz_plot(
    mask_volume, center_loc_axis, mask_bool=True, n_label=201, colorize=colorize, target_class_index=target_class_index
)
show_image(vis_mask, title="mask")

vis_image = get_xyz_plot(image_volume, center_loc_axis, mask_bool=False)
show_image(vis_image, title="image")